In [1]:
import pandas as pd
import numpy as np

from keras import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adam

import optuna

c:\Users\yveem\miniconda3\envs\machine_learning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### **Getting training & validation data**

In [2]:
df_train = pd.read_csv('../databases/training.csv')
df_valid = pd.read_csv('../databases/validation.csv')

In [3]:
X_train, y_train = df_train.drop(['IND_BOM_1_1'], axis=1), df_train['IND_BOM_1_1']
X_val, y_val = df_valid.drop(['IND_BOM_1_1'], axis=1), df_valid['IND_BOM_1_1']

In [4]:
X_train = np.array(X_train)
X_val = np.array(X_val)

#### **Parameters selection**

In [5]:
params = {
    'hidden_layer_sizes': [32, 128, 256],
    'hidden_layers': {
        'low': 1,
        'high': 2
    },
    'alpha': [0.0001, 0.01],
    'max_iter': [2000, 5000, 10000],
    'batch_size': [32, 64, 128],
    'activation': ['tanh', 'relu', 'sigmoid'],
    'optimizer': ['SGD', 'Adam'],
    'output_activation': ['sigmoid', 'softmax']
}

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    restore_best_weights=True,
    min_delta=0.001
)

def objective(trial):
    model = Sequential()
    input_dimension = X_train.shape[1]
    
    model.add(
        Dense(
            input_dim=input_dimension,
            units=trial.suggest_categorical(
                'hidden_layer_sizes',
                params['hidden_layer_sizes']
            ),
            activation=trial.suggest_categorical(
                'activation',
                params['activation']
            )
        )
    )

    hidden_layers = trial.suggest_int('hidden_layers', params['hidden_layers']['low'], params['hidden_layers']['high'])
    
    for _ in range(hidden_layers):
        model.add(
            Dense(
                units=trial.suggest_categorical(
                    'hidden_layer_sizes',
                    params['hidden_layer_sizes']
                ),
                activation=trial.suggest_categorical(
                    'activation',
                    params['activation']
                )
            )
        )
    
    model.add(
        Dense(
            units=1,
            activation=trial.suggest_categorical(
                'output_activation',
                params['output_activation']
            )
        )
    )

    alpha = trial.suggest_categorical('alpha', params['alpha'])
    optimizer_name = trial.suggest_categorical('optimizer', params['optimizer'])

    if optimizer_name == 'SGD':
        optimizer = SGD(learning_rate=alpha)
    elif optimizer_name == 'Adam':
        optimizer = Adam(learning_rate=alpha)

    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=['accuracy']
    )

    model.fit(
        X_train,
        y_train,
        batch_size=trial.suggest_categorical(
            'batch_size',
            params['batch_size']
        ),
        epochs=trial.suggest_categorical(
            'max_iter',
            params['max_iter']
        ),
        validation_data=(X_val, y_val),
        callbacks=[early_stopping]
    )

    loss, accuracy = model.evaluate(X_val, y_val)

    return loss

In [6]:
n_trials = 10

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=n_trials)

[I 2023-03-29 19:14:48,542] A new study created in memory with name: no-name-5ab2e6da-f0fc-4803-b5fa-704c8fb4d87b


Epoch 1/2000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.3445 - accuracy: 0.6555 - val_loss: 0.3446 - val_accuracy: 0.6554
Epoch 2/2000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.3445 - accuracy: 0.6555 - val_loss: 0.3446 - val_accuracy: 0.6554
Epoch 3/2000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.3445 - accuracy: 0.6555 - val_loss: 0.3446 - val_accuracy: 0.6554
Epoch 4/2000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.3445 - accuracy: 0.6555 - val_loss: 0.3446 - val_accuracy: 0.6554
Epoch 5/2000
1977/1977 [==============================] - 5s 3ms/step - loss: 0.3445 - accuracy: 0.6555 - val_loss: 0.3446 - val_accuracy: 0.6554
Epoch 6/2000
1977/1977 [==============================] - 5s 3ms/step - loss: 0.3445 - accuracy: 0.6555 - val_loss: 0.3446 - val_accuracy: 0.6554
Epoch 7/2000
1977/1977 [==============================] - 5s 2ms/step - loss: 0.3445 - accuracy: 0.6555 - val_loss: 0.3446 -

[I 2023-03-29 19:15:52,765] Trial 0 finished with value: 0.34456583857536316 and parameters: {'hidden_layer_sizes': 32, 'activation': 'relu', 'hidden_layers': 2, 'output_activation': 'softmax', 'alpha': 0.0001, 'optimizer': 'Adam', 'batch_size': 128, 'max_iter': 2000}. Best is trial 0 with value: 0.34456583857536316.


Epoch 1/10000
1977/1977 [==============================] - 10s 5ms/step - loss: 0.2376 - accuracy: 0.6078 - val_loss: 0.2315 - val_accuracy: 0.6435
Epoch 2/10000
1977/1977 [==============================] - 9s 4ms/step - loss: 0.2306 - accuracy: 0.6478 - val_loss: 0.2301 - val_accuracy: 0.6500
Epoch 3/10000
1977/1977 [==============================] - 9s 5ms/step - loss: 0.2296 - accuracy: 0.6508 - val_loss: 0.2292 - val_accuracy: 0.6513
Epoch 4/10000
1977/1977 [==============================] - 9s 4ms/step - loss: 0.2287 - accuracy: 0.6517 - val_loss: 0.2283 - val_accuracy: 0.6522
Epoch 5/10000
1977/1977 [==============================] - 9s 4ms/step - loss: 0.2279 - accuracy: 0.6522 - val_loss: 0.2275 - val_accuracy: 0.6527
Epoch 6/10000
1977/1977 [==============================] - 10s 5ms/step - loss: 0.2271 - accuracy: 0.6529 - val_loss: 0.2268 - val_accuracy: 0.6530
Epoch 7/10000
1977/1977 [==============================] - 9s 4ms/step - loss: 0.2264 - accuracy: 0.6534 - val_loss:

[I 2023-03-29 19:25:04,107] Trial 1 finished with value: 0.21507668495178223 and parameters: {'hidden_layer_sizes': 256, 'activation': 'tanh', 'hidden_layers': 2, 'output_activation': 'sigmoid', 'alpha': 0.0001, 'optimizer': 'SGD', 'batch_size': 128, 'max_iter': 10000}. Best is trial 1 with value: 0.21507668495178223.


...

[I 2023-03-29 19:37:20,204] Trial 7 finished with value: 0.34456583857536316 and parameters: {'hidden_layer_sizes': 32, 'activation': 'sigmoid', 'hidden_layers': 1, 'output_activation': 'softmax', 'alpha': 0.0001, 'optimizer': 'Adam', 'batch_size': 128, 'max_iter': 10000}. Best is trial 4 with value: 0.20473061501979828.


Epoch 1/2000
1977/1977 [==============================] - 8s 4ms/step - loss: 0.3445 - accuracy: 0.6555 - val_loss: 0.3446 - val_accuracy: 0.6554
Epoch 2/2000
1977/1977 [==============================] - 7s 4ms/step - loss: 0.3445 - accuracy: 0.6555 - val_loss: 0.3446 - val_accuracy: 0.6554
Epoch 3/2000
1977/1977 [==============================] - 8s 4ms/step - loss: 0.3445 - accuracy: 0.6555 - val_loss: 0.3446 - val_accuracy: 0.6554
Epoch 4/2000
1977/1977 [==============================] - 8s 4ms/step - loss: 0.3445 - accuracy: 0.6555 - val_loss: 0.3446 - val_accuracy: 0.6554
Epoch 5/2000
1977/1977 [==============================] - 7s 4ms/step - loss: 0.3445 - accuracy: 0.6555 - val_loss: 0.3446 - val_accuracy: 0.6554
Epoch 6/2000
1977/1977 [==============================] - 7s 4ms/step - loss: 0.3445 - accuracy: 0.6555 - val_loss: 0.3446 - val_accuracy: 0.6554
Epoch 7/2000
1977/1977 [==============================] - 7s 4ms/step - loss: 0.3445 - accuracy: 0.6555 - val_loss: 0.3446 -

[I 2023-03-29 19:38:48,913] Trial 8 finished with value: 0.34456583857536316 and parameters: {'hidden_layer_sizes': 256, 'activation': 'sigmoid', 'hidden_layers': 1, 'output_activation': 'softmax', 'alpha': 0.01, 'optimizer': 'Adam', 'batch_size': 128, 'max_iter': 2000}. Best is trial 4 with value: 0.20473061501979828.


Epoch 1/2000
3953/3953 [==============================] - 8s 2ms/step - loss: 0.2480 - accuracy: 0.5390 - val_loss: 0.2387 - val_accuracy: 0.6554
Epoch 2/2000
3953/3953 [==============================] - 7s 2ms/step - loss: 0.2340 - accuracy: 0.6555 - val_loss: 0.2307 - val_accuracy: 0.6554
Epoch 3/2000
3953/3953 [==============================] - 7s 2ms/step - loss: 0.2289 - accuracy: 0.6555 - val_loss: 0.2277 - val_accuracy: 0.6554
Epoch 4/2000
3953/3953 [==============================] - 7s 2ms/step - loss: 0.2270 - accuracy: 0.6555 - val_loss: 0.2266 - val_accuracy: 0.6554
Epoch 5/2000
3953/3953 [==============================] - 7s 2ms/step - loss: 0.2263 - accuracy: 0.6555 - val_loss: 0.2261 - val_accuracy: 0.6554
Epoch 6/2000
3953/3953 [==============================] - 8s 2ms/step - loss: 0.2260 - accuracy: 0.6555 - val_loss: 0.2259 - val_accuracy: 0.6554
Epoch 7/2000
3953/3953 [==============================] - 8s 2ms/step - loss: 0.2259 - accuracy: 0.6555 - val_loss: 0.2258 -

[I 2023-03-29 19:40:38,216] Trial 9 finished with value: 0.2265513390302658 and parameters: {'hidden_layer_sizes': 32, 'activation': 'sigmoid', 'hidden_layers': 2, 'output_activation': 'sigmoid', 'alpha': 0.0001, 'optimizer': 'SGD', 'batch_size': 64, 'max_iter': 2000}. Best is trial 4 with value: 0.20473061501979828.


In [7]:
print('Best hyperparameters: ', study.best_params)
print('Best objective value: ', study.best_value)

Best hyperparameters:  {'hidden_layer_sizes': 32, 'activation': 'sigmoid', 'hidden_layers': 2, 'output_activation': 'sigmoid', 'alpha': 0.0001, 'optimizer': 'Adam', 'batch_size': 64, 'max_iter': 5000}
Best objective value:  0.20473061501979828


#### **Saving study**

In [8]:
import joblib

In [9]:
save_path = './optuna_studies/mlp_study.pkl'

joblib.dump(study, save_path)

['./optuna_studies/mlp_study.pkl']